In [29]:
import pandas as pd
import os

df = pd.read_json('../Data/Serie_A_2014_squads.json')

df = df.T
df

,0,1,2,3,4,5,6,7,8
1,"{'Home': {'Squad': ['141698', '137644', '14474...","{'Home': {'Squad': ['141698', '137644', '14474...","{'Home': {'Squad': ['141698', '137644', '14474...","{'Home': {'Squad': ['141698', '137644', '14474...","{'Home': {'Squad': ['141698', '137644', '14474...","{'Home': {'Squad': ['141698', '137644', '14474...",NaN,NaN,NaN
2,"{'Home': {'Squad': ['137990', '172996', '16795...","{'Home': {'Squad': ['137990', '172996', '16795...","{'Home': {'Squad': ['137990', '172996', '16795...","{'Home': {'Squad': ['137990', '172996', '16795...","{'Home': {'Squad': ['137990', '172996', '16795...","{'Home': {'Squad': ['137990', '172996', '16795...",NaN,NaN,NaN
3,"{'Home': {'Squad': ['107639', '161940', '45966...","{'Home': {'Squad': ['107639', '161940', '45966...","{'Home': {'Squad': ['107639', '161940', '45966...","{'Home': {'Squad': ['107639', '161940', '31664...",NaN,NaN,NaN,NaN,NaN
4,"{'Home': {'Squad': ['142720', '185679', '16194...","{'Home': {'Squad': ['142720', '185679', '16194...","{'Home': {'Squad': ['142720', '185679', '16194...","{'Home': {'Squad': ['142720', '185679', '33783...","{'Home': {'Squad': ['142720', '185679', '33783...","{'Home': {'Squad': ['142720', '185679', '33783...",NaN,NaN,NaN
5,"{'Home': {'Squad': ['293428', '322380', '18260...","{'Home': {'Squad': ['293428', '322380', '18260...","{'Home': {'Squad': ['293428', '322380', '18260...","{'Home': {'Squad': ['293428', '322380', '18260...","{'Home': {'Squad': ['293428', '322380', '18260...","{'Home': {'Squad': ['293428', '322380', '18260...","{'Home': {'Squad': ['293428', '322380', '18260...",NaN,NaN
...,...,...,...,...,...,...,...,...,...
376,"{'Home': {'Squad': ['180213', '174360', '15728...","{'Home': {'Squad': ['180213', '174360', '15728...","{'Home': {'Squad': ['180213', '174360', '15728...","{'Home': {'Squad': ['180213', '174360', '15728...","{'Home': {'Squad': ['180213', '174360', '15728...","{'Home': {'Squad': ['180213', '174360', '15728...","{'Home': {'Squad': ['180213', '174360', '15728...",NaN,NaN
377,"{'Home': {'Squad': ['511533', '169529', '29338...","{'Home': {'Squad': ['511533', '169529', '29338...","{'Home': {'Squad': ['511533', '169529', '29338...","{'Home': {'Squad': ['511533', '169529', '29338...","{'Home': {'Squad': ['511533', '169529', '29338...","{'Home': {'Squad': ['511533', '169529', '29338...","{'Home': {'Squad': ['511533', '169529', '29338...",NaN,NaN
378,"{'Home': {'Squad': ['168351', '165065', '16928...","{'Home': {'Squad': ['168351', '165065', '16928...","{'Home': {'Squad': ['168351', '165065', '16928...","{'Home': {'Squad': ['168351', '165065', '16928...","{'Home': {'Squad': ['168351', '165065', '16928...","{'Home': {'Squad': ['168351', '165065', '16928...","{'Home': {'Squad': ['168351', '165065', '16928...",NaN,NaN
379,"{'Home': {'Squad': ['291584', '171788', '16103...","{'Home': {'Squad': ['291584', '171788', '16103...","{'Home': {'Squad': ['291584', '171788', '16103...","{'Home': {'Squad': ['291584', '171788', '16103...","{'Home': {'Squad': ['291584', '171788', '16103...","{'Home': {'Squad': ['291584', '171788', '16103...","{'Home': {'Squad': ['291584', '171788', '16103...","{'Home': {'Squad': ['291584', '171788', '16103...","{'Home': {'Squad': ['291584', '161039', '30731..."


Vamos pegar as informações dos ID's dos jogadores com as informações dos Squads. Começaremos pelo campeonato da Série A de 2013 até 2024, e nossa intenção é definir um rating inicial de 1500 para cada jogador.

Com isso, conseguiremos calcular um rating de uma partida e atualizar o rating dos jogadores após ela nesse dataframe com os ID's.

In [30]:

def collect_unique_ids(data_folder, file_names, years):
    """
    Função para coletar IDs únicos de jogadores em arquivos JSON.

    :param data_folder: Caminho para o diretório onde os arquivos JSON estão localizados.
    :param file_names: Lista de nomes de arquivos base (ex: ['Serie_A', 'Serie_B', 'Copa_do_Brasil']).
    :param years: Lista ou range de anos dos arquivos a serem processados.
    :return: Lista de IDs únicos coletados de todos os arquivos.
    """

    unique_ids = set()

    def process_dataframe(df):
        for column in df.columns:
            for cell in df[column]:
                if isinstance(cell, dict):  
                    for key in ['Home', 'Away']:
                        team_info = cell.get(key, {})
                        # Adicionar os IDs do 'Squad'
                        squad_ids = team_info.get('Squad', [])
                        unique_ids.update(squad_ids)


    for file_name in file_names:
        for year in years:
            file_path = os.path.join(data_folder, f'{file_name}_{year}_squads.json')
            if os.path.exists(file_path):
                df = pd.read_json(file_path)
                df_ = df.iloc[:,1]
                df = pd.DataFrame(df_)
                df.to_excel('df.xlsx')
                process_dataframe(df)
            else:
                print(f'Arquivo não encontrado: {file_path}')


    return list(unique_ids)

data_folder = '../Data/'
#file_names = ['Serie_A', 'Serie_B', 'Serie_C', 'Serie_D', 'CdB']
file_names = ['Serie_A']
years = range(2014, 2015)

unique_player_ids = collect_unique_ids(data_folder, file_names, years)

print(f'Total de IDs únicos coletados: {len(unique_player_ids)}')

Total de IDs únicos coletados: 28


Total de 15448 diferentes jogadores registrados!

Vamos trabalhar com os 3157 da série A. Simularemos inicialmente idades e jogos jogados.

In [31]:
df_players = pd.DataFrame(unique_player_ids, columns=['player'])

# Adicionar a coluna 'rating' com valor inicial de 1500 para cada jogador
df_players['rating'] = 1500
df_players['age'] = 25
df_players['games_played'] = 50

df_players.to_excel('id_players.xlsx')

# Mostrar o DataFrame criado
df_players

,player,rating,age,games_played
0,184931,1500,25,50
1,173180,1500,25,50
2,178723,1500,25,50
3,295587,1500,25,50
4,358318,1500,25,50
5,165064,1500,25,50
6,150549,1500,25,50
7,175560,1500,25,50
8,180884,1500,25,50
9,324030,1500,25,50
